# IMPORTING DATA AND CLEANING PROCESS

### This jupyter file contains the most important part of the data cleaning, wrangling and preparation. The core of the process lies on 3 axis: 
- The import of FIFA ratings csv file
- The import of our desired years' Ballon d'Or classifications
- The mixing process to filter FIFA ratings according to the information in obtained through web scrapping.

In [1]:
# First, we import the necessary libraries:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import sys

In [2]:
sys.path.append('../py_files') # <-- relative path
import functions

The process starts by importing and reading the csv file with historical FIFA ratings, recorded from 2015-2016 season to 2022-2023.

There are plenty of players, the dataframe has data of +160,000 player versions, which is a huge amount. However, we will focus on the top 10 players according to each year Ballon d'Or classification from 2017 (the last 5 editions).

To do so, we will divide the dataframe by years, from 2017 to 2022, without 2020, year in which Ballon d'Or was not awarded due to COVID pandemic.

In [3]:
def csv_reader (file):
    return pd.read_csv(file, low_memory = False)
     
fifa_ratings = csv_reader("../csv_files/male_players (legacy).csv")

In [4]:
# First, we define the function that will iterate through every column of the dataset asking us if we want to keep it or not:
def column_droper(df):
    for column in df.columns:
        column_keeper = "--"
        continuer = "--"
        while len(column_keeper) != 1 and column_keeper not in ["Y", "N"]:
            column_keeper = input(f"Do you want to keep column {column}? (Y/N)")
            if column_keeper == "Y":
                column_keeper = "--"
                break
            elif column_keeper == "N":
                df.drop([column], axis = 1, inplace = True)
                column_keeper = "--"
                break
            else:
                column_keeper = "--"

In [5]:
fifa_ratings

,player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url
0,158023,/player/158023/lionel-messi/150002,15,2,2014-09-18,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,95,...,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,https://cdn.sofifa.net/players/158/023/15_120.png
1,20801,/player/20801/c-ronaldo-dos-santos-aveiro/150002,15,2,2014-09-18,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",92,92,...,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3,https://cdn.sofifa.net/players/020/801/15_120.png
2,9014,/player/9014/arjen-robben/150002,15,2,2014-09-18,A. Robben,Arjen Robben,"RM, LM, RW",90,90,...,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3,https://cdn.sofifa.net/players/009/014/15_120.png
3,41236,/player/41236/zlatan-ibrahimovic/150002,15,2,2014-09-18,Z. Ibrahimović,Zlatan Ibrahimović,ST,90,90,...,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3,https://cdn.sofifa.net/players/041/236/15_120.png
4,167495,/player/167495/manuel-neuer/150002,15,2,2014-09-18,M. Neuer,Manuel Peter Neuer,GK,90,90,...,40+3,40+3,36+3,36+3,38+3,38+3,38+3,36+3,87+3,https://cdn.sofifa.net/players/167/495/15_120.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,269011,/player/269011/ziye-zhao/230002,23,2,2022-09-26,Zhao Ziye,赵梓业,RW,46,61,...,32+2,32+2,37+2,36+2,31+2,31+2,31+2,36+2,14+2,https://cdn.sofifa.net/players/269/011/23_120.png
161579,269019,/player/269019/junjie-wu/230002,23,2,2022-09-26,Wu Junjie,吴俊杰,LB,46,58,...,41+2,41+2,43+2,44+2,48+2,48+2,48+2,44+2,15+2,https://cdn.sofifa.net/players/269/019/23_120.png
161580,271093,/player/271093/kailin-barlow/230002,23,2,2022-09-26,K. Barlow,Kailin Barlow,CDM,46,58,...,45+2,45+2,47+2,47+2,46+2,46+2,46+2,47+2,14+2,https://cdn.sofifa.net/players/271/093/23_120.png
161581,271555,/player/271555/gideon-tetteh/230002,23,2,2022-09-26,G. Tetteh,Gideon Tetteh,"RW, LW",46,70,...,30+2,30+2,34+2,32+2,28+2,28+2,28+2,32+2,15+2,https://cdn.sofifa.net/players/271/555/23_120.png


In [6]:
# By applying the function, we choose the variables that are of our interest:
# column_droper(fifa_ratings)

In [7]:
# Our variables of interest are the following
variables_of_interest = (list(fifa_ratings.columns))
length = len(variables_of_interest)
print(f"We maintain {length} variables out of the original 110: {variables_of_interest}")

We maintain 110 variables out of the original 110: ['player_id', 'player_url', 'fifa_version', 'fifa_update', 'fifa_update_date', 'short_name', 'long_name', 'player_positions', 'overall', 'potential', 'value_eur', 'wage_eur', 'age', 'dob', 'height_cm', 'weight_kg', 'league_id', 'league_name', 'league_level', 'club_team_id', 'club_name', 'club_position', 'club_jersey_number', 'club_loaned_from', 'club_joined_date', 'club_contract_valid_until_year', 'nationality_id', 'nationality_name', 'nation_team_id', 'nation_position', 'nation_jersey_number', 'preferred_foot', 'weak_foot', 'skill_moves', 'international_reputation', 'work_rate', 'body_type', 'real_face', 'release_clause_eur', 'player_tags', 'player_traits', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic', 'attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy', 'attacking_short_passing', 'attacking_volleys', 'skill_dribbling', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_con

In [8]:
# To avoid run the function above every time we reset the kernel, we store the columns in a list
# Also, we create the dataset that we will work on.
sub_dataset = ['player_id', 'fifa_version', 'short_name', 'long_name', 'player_positions', 'overall', 'value_eur', 'wage_eur', 'age', 'height_cm', 'weight_kg', 'league_name', 'club_name', 'club_position', 'club_jersey_number', 'nationality_name', 'preferred_foot', 'release_clause_eur', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic', 'goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_kicking', 'goalkeeping_positioning', 'goalkeeping_reflexes', 'goalkeeping_speed']
ratings = fifa_ratings[(sub_dataset)]

In [9]:
ratings

,player_id,fifa_version,short_name,long_name,player_positions,overall,value_eur,wage_eur,age,height_cm,...,passing,dribbling,defending,physic,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
0,158023,15,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,100500000.0,550000.0,27,169,...,86.0,96.0,27.0,63.0,6,11,15,14,8,NaN
1,20801,15,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",92,79000000.0,375000.0,29,185,...,81.0,91.0,32.0,79.0,7,11,15,14,11,NaN
2,9014,15,A. Robben,Arjen Robben,"RM, LM, RW",90,54500000.0,275000.0,30,180,...,83.0,92.0,32.0,64.0,10,8,11,5,15,NaN
3,41236,15,Z. Ibrahimović,Zlatan Ibrahimović,ST,90,52500000.0,275000.0,32,195,...,81.0,86.0,34.0,86.0,13,15,10,9,12,NaN
4,167495,15,M. Neuer,Manuel Peter Neuer,GK,90,63500000.0,300000.0,28,193,...,NaN,NaN,NaN,NaN,87,85,92,90,86,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,269011,23,Zhao Ziye,赵梓业,RW,46,110000.0,700.0,18,180,...,40.0,47.0,24.0,44.0,9,13,13,12,7,NaN
161579,269019,23,Wu Junjie,吴俊杰,LB,46,110000.0,750.0,19,188,...,29.0,34.0,48.0,57.0,6,14,8,13,14,NaN
161580,271093,23,K. Barlow,Kailin Barlow,CDM,46,110000.0,500.0,19,181,...,43.0,46.0,43.0,53.0,8,9,7,14,9,NaN
161581,271555,23,G. Tetteh,Gideon Tetteh,"RW, LW",46,150000.0,500.0,17,175,...,36.0,46.0,20.0,42.0,13,12,14,7,13,NaN


In [10]:
# Now that our dataframe contains only columns that are useful for our study, we apply data filters.
# First, we divide the dataset by each year of interest. Also, we already define 
# a url with a link to wikipedia, from where we are going to scrap the Ballon d'Or classification
# for that given year.

def year_filter (df, year):
    """
    Function that filters FIFA ratings' dataframe by FIFA version
    Receives the dataframe and a year input, which must be in two digits format (YY), e.g.
    17 for 2017.
    Returns the filtered version of the dataframe by year.
    """
    df = df[(df["fifa_version"] == year)]
    df.reset_index(drop = True, inplace = True)
    return df

fifa_ratings_2017 = year_filter(ratings, 17)
url_2017 = "https://en.wikipedia.org/wiki/2017_Ballon_d%27Or"

fifa_ratings_2018 = year_filter(ratings, 18)
url_2018 = "https://en.wikipedia.org/wiki/2018_Ballon_d%27Or"

fifa_ratings_2019 = year_filter(ratings, 19)
url_2019 = "https://en.wikipedia.org/wiki/2019_Ballon_d%27Or"

fifa_ratings_2021 = year_filter(ratings, 21)
url_2021 = "https://en.wikipedia.org/wiki/2021_Ballon_d%27Or"

fifa_ratings_2022 = year_filter(ratings, 22)
url_2022 = "https://en.wikipedia.org/wiki/2022_Ballon_d%27Or"

In [11]:
# We will run the process once to make sure that the extraction works well, e.g. with 2017:
url_2017 = "https://en.wikipedia.org/wiki/2017_Ballon_d%27Or"
html = requests.get(url_2017)
soup = BeautifulSoup(html.content, "html.parser")
tags = soup.find_all("table", attrs = {"class":"wikitable"}) # <table class="wikitable plainrowheaders sortable jquery-tablesorter">
classification_2017 = pd.read_html(tags[0].prettify())[0][:10]
classification_2017

,Rank,Player,Club(s),Points
0,1,Cristiano Ronaldo,Real Madrid,946
1,2,Lionel Messi,Barcelona,670
2,3,Neymar,Barcelona Paris Saint-Germain,361
3,4,Gianluigi Buffon,Juventus,221
4,5,Luka Modrić,Real Madrid,84
5,6,Sergio Ramos,Real Madrid,71
6,7,Kylian Mbappé,Monaco Paris Saint-Germain,48
7,8,N'Golo Kanté,Chelsea,47
8,9,Robert Lewandowski,Bayern Munich,45
9,10,Harry Kane,Tottenham Hotspur,36


In [12]:
def get_year_classification(url):
    """
    Function that provides a dataset with the classification of the Ballon d'Or.
    Accepts an url as argument, which must be the wikipedia page for a given year Ballon d'Or.
    Returns a dataset with the ranking, name club and points obtained by top 10 players.
    """
    html = requests.get(url)
    soup = BeautifulSoup(html.content, "html.parser")
    tags = soup.find_all("table", attrs = {"class":"wikitable"}) # <table class="wikitable">
    classification = pd.read_html(tags[0].prettify())[0][:10]
    return classification

In [13]:
classification_2017 = get_year_classification(url_2017)
classification_2018 = get_year_classification(url_2018)
classification_2019 = get_year_classification(url_2019)
classification_2021 = get_year_classification(url_2021)
classification_2022 = get_year_classification(url_2022)

In [14]:
# For 2018, there is a note in Luka Modric's points. We clean it manually since its fast
# and only one case among all 5 classification datasets:
(classification_2018[(classification_2018['Player'] == "Luka Modrić")]['Points'][0])

'753  [6]'

In [15]:
classification_2018['Points'][0] = 753
classification_2018

,Rank,Player,Club(s),Points
0,1,Luka Modrić,Real Madrid,753
1,2,Cristiano Ronaldo,Real Madrid Juventus,478
2,3,Antoine Griezmann,Atlético Madrid,414
3,4,Kylian Mbappé,Paris Saint-Germain,347
4,5,Lionel Messi,Barcelona,280
5,6,Mohamed Salah,Liverpool,188
6,7,Raphaël Varane,Real Madrid,121
7,8,Eden Hazard,Chelsea,119
8,9,Kevin De Bruyne,Manchester City,29
9,10,Harry Kane,Tottenham Hotspur,25


We have just obtained a dataset with the best 10 football players from last five Ballon d'Or editions. Now, we want to filter the FIFA ratings of each of the years to get data only from the top 10 players of that given year.

To do so, we will create another function that will put the names of our players in a list.

In [16]:
def top_10_players (df):
    """
    Function that gives a list with the names of the top10 players
    From a given year classification dataframe
    """
    return [player for player in df["Player"]]

In [17]:
# An example of the function running well:
top_10_players(classification_2022)

['Karim Benzema',
 'Sadio Mané',
 'Kevin De Bruyne',
 'Robert Lewandowski',
 'Mohamed Salah',
 'Kylian Mbappé',
 'Thibaut Courtois',
 'Vinícius Júnior',
 'Luka Modrić',
 'Erling Haaland']

In [18]:
top_10_2017 = top_10_players(classification_2017)
top_10_2018 = top_10_players(classification_2018)
top_10_2019 = top_10_players(classification_2019)
top_10_2021 = top_10_players(classification_2021)
top_10_2022 = top_10_players(classification_2022)


The objective of this previous steps is to get 5 different dataframes, one for each year.
Each dataframe must contain FIFA ratings and stats from the lists of top10 players that we've just obtained.

Before getting into this task, we get a list with all players in the 5 top 10s to make a regex for each of them, so we can match them to the long_name column in the FIFA ratings database.

Then, the only step left is to filter dataframes by each players' list. We will build a function that allows us to achieve the objective.

In [19]:
def all_players ():
    all_players_list = set(top_10_2017 + top_10_2018 + top_10_2019 + top_10_2021 + top_10_2022)
    return all_players_list

In [20]:
# We create the list containing all unique players' name:
all_players_list = all_players()

In [21]:
print(all_players_list)

{'Bernardo Silva', "N'Golo Kanté", 'Thibaut Courtois', 'Alisson', 'Sadio Mané', 'Antoine Griezmann', 'Raphaël Varane', 'Karim Benzema', 'Vinícius Júnior', 'Virgil van Dijk', 'Robert Lewandowski', 'Gianluigi Buffon', 'Luka Modrić', 'Eden Hazard', 'Jorginho', 'Harry Kane', 'Sergio Ramos', 'Neymar', 'Gianluigi Donnarumma', 'Lionel Messi', 'Mohamed Salah', 'Cristiano Ronaldo', 'Kylian Mbappé', 'Kevin De Bruyne', 'Riyad Mahrez', 'Erling Haaland'}


In [22]:
# '.*Cristiano.*Ronaldo.*'
def regex_generator (players_list):
    """
    Function that creates a regular expression for each unique player in all top10's list
    Gets the list of unique players' names as argument
    Returns a dictionary where keys are players' names and values are regex that can match
    their name in FIFA ratings dataframe
    """
    regex_dict = {}
    
    for player in players_list:
        regex_dict[player] = (".*" + ".*".join(player.split(" ")) + ".*")

    return regex_dict

In [23]:
regex_dict = regex_generator(all_players_list)
regex_dict

{'Bernardo Silva': '.*Bernardo.*Silva.*',
 "N'Golo Kanté": ".*N'Golo.*Kanté.*",
 'Thibaut Courtois': '.*Thibaut.*Courtois.*',
 'Alisson': '.*Alisson.*',
 'Sadio Mané': '.*Sadio.*Mané.*',
 'Antoine Griezmann': '.*Antoine.*Griezmann.*',
 'Raphaël Varane': '.*Raphaël.*Varane.*',
 'Karim Benzema': '.*Karim.*Benzema.*',
 'Vinícius Júnior': '.*Vinícius.*Júnior.*',
 'Virgil van Dijk': '.*Virgil.*van.*Dijk.*',
 'Robert Lewandowski': '.*Robert.*Lewandowski.*',
 'Gianluigi Buffon': '.*Gianluigi.*Buffon.*',
 'Luka Modrić': '.*Luka.*Modrić.*',
 'Eden Hazard': '.*Eden.*Hazard.*',
 'Jorginho': '.*Jorginho.*',
 'Harry Kane': '.*Harry.*Kane.*',
 'Sergio Ramos': '.*Sergio.*Ramos.*',
 'Neymar': '.*Neymar.*',
 'Gianluigi Donnarumma': '.*Gianluigi.*Donnarumma.*',
 'Lionel Messi': '.*Lionel.*Messi.*',
 'Mohamed Salah': '.*Mohamed.*Salah.*',
 'Cristiano Ronaldo': '.*Cristiano.*Ronaldo.*',
 'Kylian Mbappé': '.*Kylian.*Mbappé.*',
 'Kevin De Bruyne': '.*Kevin.*De.*Bruyne.*',
 'Riyad Mahrez': '.*Riyad.*Mahrez.*

In [24]:
# Let's test if regex expressions actually match long names in ratings dataframe:
cristiano = fifa_ratings_2017["long_name"][0]
regex_cristiano = regex_dict["Cristiano Ronaldo"]
matched = re.match(regex_cristiano, cristiano)
is_match = bool(matched)
is_match # Prints True, which feels amazing; now we know that we can filter using regex_dict!

True

Let's try to build the function that gives us the cleaned dataframe with only top10 footballers FIFA ratings from the list of the classification and the regex_dict.

In [25]:
def long_name_creator (df, top10_year):
    long_name_list = []
    for player in top10_year:
        for long_name in df["long_name"]:
            if re.match(regex_dict[player], long_name):
                long_name_list.append(long_name)
            else:
                pass
    
    return list(set(long_name_list))

In [26]:
# We have some issues with some players' regex: 
# Jorginho, Allison, Thibaut Courtios, Eden Hazard, Mohamed Salah and Bernardo Silva.
# Since it is a low number of cases, we fix them manually in the regex_dict.
regex_dict["Jorginho"] = "Luiz Frello Filho Jorge"
regex_dict["Bernardo Silva"] = "Bernardo Mota Veiga de Carvalho e Silva"
regex_dict["Eden Hazard"] = "Eden Hazard"
regex_dict["Mohamed Salah"] = "Mohamed Salah Ghaly"
regex_dict["Thibaut Courtois"] = "Thibaut Nicolas Marc Courtois"
regex_dict["Alisson"] = "Alisson Ramsés Becker"

In [27]:
longnames_2017 = long_name_creator(fifa_ratings_2017, top_10_2017)
longnames_2018 = long_name_creator(fifa_ratings_2018, top_10_2018)
longnames_2019 = long_name_creator(fifa_ratings_2019, top_10_2019)
longnames_2021 = long_name_creator(fifa_ratings_2021, top_10_2021)
longnames_2022 = long_name_creator(fifa_ratings_2022, top_10_2022)

In [28]:
len(longnames_2021) == len(top_10_2021) # now both lists should have the same length and players_filter will run smoothly!

True

In [29]:
# An example of a long_names list for year 2019:
print(longnames_2019)

['Kylian Mbappé Lottin', 'Lionel Andrés Messi Cuccittini', 'Mohamed Salah Ghaly', 'Bernardo Mota Veiga de Carvalho e Silva', 'Alisson Ramsés Becker', 'Sadio Mané', 'Riyad Mahrez', 'Virgil van Dijk', 'Robert Lewandowski', 'Cristiano Ronaldo dos Santos Aveiro']


In [30]:
def players_filter (df, longnames_list):
    """
    Function that filters the original yearly dataframe applying name conditions on long_name column.
    It accepts two arguments.
        First, the dataframe to filter, e.g. fifa_ratings_2017.
        Second, the list of long_names obtained through the iteration of the regex dict and
        the list with the names of the year's Ballon d'Or classification.
    It returns the dataframe with the FIFA ratings of that particular year best players. 
    """
    c_plr_1 = (df["long_name"] == longnames_list[0]) # c_plr_1 stands for condition_player_1
    c_plr_2 = (df["long_name"] == longnames_list[1])
    c_plr_3 = (df["long_name"] == longnames_list[2])
    c_plr_4 = (df["long_name"] == longnames_list[3])
    c_plr_5 = (df["long_name"] == longnames_list[4])
    c_plr_6 = (df["long_name"] == longnames_list[5])
    c_plr_7 = (df["long_name"] == longnames_list[6])
    c_plr_8 = (df["long_name"] == longnames_list[7])
    c_plr_9 = (df["long_name"] == longnames_list[8])
    c_plr_10 = (df["long_name"] == longnames_list[9])
    
    dataframe = df[(c_plr_1)|(c_plr_2)|(c_plr_3)|(c_plr_4)|(c_plr_5)|(c_plr_6)|(c_plr_7)|(c_plr_8)|(c_plr_9)|(c_plr_10)]
    return dataframe
      

In [31]:
fifa_ratings_2017 = players_filter(fifa_ratings_2017, longnames_2017)
fifa_ratings_2018 = players_filter(fifa_ratings_2018, longnames_2018)
fifa_ratings_2019 = players_filter(fifa_ratings_2019, longnames_2019)
fifa_ratings_2021 = players_filter(fifa_ratings_2021, longnames_2021)
fifa_ratings_2022 = players_filter(fifa_ratings_2022, longnames_2022)

So far, we have made some important progress.
We have been able to get a csv file with players FIFA stats, which we have subdivided in 5 different seasons: 2017, 2018, 2019, 2021 and 2022. In addition, we have removed the columns that are not of our interest.

Also, we have scrapped those years' Ballon d'Or standings, with the objective to use the names in each year final results to filter the divided subsets.

We have made this filtering through a dictionary in which we have stored key/value pairs of players' name and it's regular expression that could match the 'long_name' column in the FIFA ratings dataframes. Eventually, we managed to improve the usability of each year's classifications by modifying the players' names, giving them the same format as in 'long_name' column. Next step was to apply some conditions to each year dataframe, to reduce them to store only our 10 preferred players.

And last but not least, we introduced the punctuation that each player received for every edition of the Ballon d'Or ceremony.

## Appending Ballon d'Or POINTS for each player in FIFA ratings dataframes.

FIFA ratings dataframes are sorted by overall players' ratings, not by that years' Ballon d'Or classification. Hence, to append the punctuation into its correct player we will have to create a pivot dataframe where we can mix the name of players, its rating and the points, in order to sort it by rating and make it compatible with FIFA ratings datasets. 

In [32]:
# To have the same index starting from 0 in every dataset:
def index_reset (df):
    df.reset_index(drop = True, inplace = True)
    return df

In [33]:
fifa_ratings_2017 = index_reset(fifa_ratings_2017)
fifa_ratings_2018 = index_reset(fifa_ratings_2018)
fifa_ratings_2019 = index_reset(fifa_ratings_2019)
fifa_ratings_2021 = index_reset(fifa_ratings_2021)
fifa_ratings_2022 = index_reset(fifa_ratings_2022)

In [34]:
fifa_ratings_2022

,player_id,fifa_version,short_name,long_name,player_positions,overall,value_eur,wage_eur,age,height_cm,...,passing,dribbling,defending,physic,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
0,188545,22,R. Lewandowski,Robert Lewandowski,ST,92,119500000.0,270000.0,32,185,...,79.0,86.0,44.0,82.0,15,6,12,8,10,NaN
1,192985,22,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,125500000.0,350000.0,30,181,...,93.0,88.0,64.0,78.0,15,13,5,10,13,NaN
2,231747,22,K. Mbappé,Kylian Mbappé Lottin,"ST, LW",91,194000000.0,230000.0,22,182,...,80.0,92.0,36.0,77.0,13,5,7,11,6,NaN
3,165153,22,K. Benzema,Karim Benzema,"CF, ST",89,66000000.0,350000.0,33,185,...,81.0,87.0,39.0,77.0,13,11,5,5,7,NaN
4,192119,22,T. Courtois,Thibaut Nicolas Marc Courtois,GK,89,85500000.0,250000.0,29,199,...,NaN,NaN,NaN,NaN,84,89,74,86,88,46.0
5,208722,22,S. Mané,Sadio Mané,LW,89,101000000.0,270000.0,29,175,...,80.0,89.0,44.0,77.0,10,10,15,7,14,NaN
6,209331,22,M. Salah,Mohamed Salah Ghaly,RW,89,101000000.0,270000.0,29,175,...,81.0,90.0,45.0,75.0,14,14,9,11,14,NaN
7,239085,22,E. Haaland,Erling Braut Haaland,ST,88,137500000.0,110000.0,20,194,...,65.0,80.0,45.0,88.0,7,14,13,11,7,NaN
8,177003,22,L. Modrić,Luka Modrić,CM,87,32000000.0,190000.0,35,172,...,89.0,88.0,72.0,66.0,13,9,7,14,9,NaN
9,238794,22,Vinícius Jr.,Vinícius José Paixão de Oliveira Júnior,LW,80,46500000.0,120000.0,20,176,...,71.0,87.0,29.0,66.0,5,7,7,7,10,NaN


A minor cleaning step is to keep only one position per player, removing those playing roles that are not that common.

In [35]:
def single_position (df):
    """
    Function that removes non-common playing positions for each player.
    It accepts FIFA ratings of a given year.
    Returns it with players having only one position attribute, the most common, e.g.
    striker, left wing, etc.
    """
    for i in range(df.shape[0]):
        df["player_positions"][i] = df["player_positions"][i].split(",")[0]
    
    return df

In [36]:
fifa_ratings_2017 = single_position(fifa_ratings_2017)
fifa_ratings_2018 = single_position(fifa_ratings_2018)
fifa_ratings_2019 = single_position(fifa_ratings_2019)
fifa_ratings_2021 = single_position(fifa_ratings_2021)
fifa_ratings_2022 = single_position(fifa_ratings_2022)

/var/folders/lr/stnkft6j5ps6pysrw7v0cgp80000gn/T/ipykernel_4799/30412018.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["player_positions"][i] = df["player_positions"][i].split(",")[0]


In [37]:
# We can see that player_positions only have one element now!
fifa_ratings_2017

,player_id,fifa_version,short_name,long_name,player_positions,overall,value_eur,wage_eur,age,height_cm,...,passing,dribbling,defending,physic,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
0,20801,17,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,LW,94,87000000.0,575000.0,31,185,...,81.0,91.0,33.0,80.0,7,11,15,14,11,NaN
1,158023,17,L. Messi,Lionel Andrés Messi Cuccittini,RW,93,89000000.0,575000.0,29,170,...,86.0,96.0,26.0,61.0,6,11,15,14,8,NaN
2,190871,17,Neymar,Neymar da Silva Santos Júnior,LW,92,109000000.0,450000.0,24,174,...,78.0,95.0,30.0,56.0,9,9,15,15,11,NaN
3,188545,17,R. Lewandowski,Robert Lewandowski,ST,90,71000000.0,250000.0,27,185,...,74.0,85.0,38.0,82.0,15,6,12,8,10,NaN
4,155862,17,Sergio Ramos,Sergio Ramos García,CB,89,43500000.0,300000.0,30,183,...,70.0,71.0,87.0,83.0,11,8,9,7,11,NaN
5,177003,17,L. Modrić,Luka Modrić,CM,89,54000000.0,325000.0,30,174,...,86.0,89.0,72.0,66.0,13,9,7,14,9,NaN
6,1179,17,G. Buffon,Gianluigi Buffon,GK,88,9000000.0,120000.0,38,192,...,NaN,NaN,NaN,NaN,87,88,68,90,84,46.0
7,202126,17,H. Kane,Harry Kane,ST,84,41000000.0,140000.0,22,188,...,71.0,78.0,42.0,81.0,8,10,11,14,11,NaN
8,215914,17,N. Kanté,N'Golo Kanté,CM,81,21000000.0,130000.0,25,169,...,74.0,77.0,81.0,83.0,15,12,10,7,10,NaN
9,231747,17,K. Mbappe Lottin,Kylian Mbappé Lottin,LW,71,3100000.0,20000.0,17,178,...,65.0,74.0,39.0,62.0,13,5,7,11,6,NaN


### Points appender function

Down below, we define the most important function of the process so far, the one that will allow us to append the punctuation to each dataframe just by introducing each year FIFA ratings, top10 and full classifications.

In [38]:
def points_column_append (df, top10_year, classification):
    """
    This amazing function has a very important objective: append the points 
    received by each player in a Ballon d'Or ceremony to the eventual yearly FIFA
    ratings dataframe. This is really important because points are the result to which
    we will want to run regressions and stuff!

    It receives the yearly FIFA ratings dataframe, the top 10 players of that year, and the 
    scrapped classification.
    
    It returns the FIFA ratings dataframe with brand new points column!!! (Struggled a lot here)
    """
    info_dict = {"long_name": [], "fifa_index": [], "rating": [], "points": [], "short_name": []}
    
    for player in top10_year:
        for long_name in df["long_name"]:
            if re.match(regex_dict[player], long_name):
                info_dict["long_name"].append(long_name)
            else:
                pass
    
    for player in info_dict["long_name"]:
        info_dict['fifa_index'].append(df[(df['long_name'] == player)].index[0])
    
    for i in range(len(info_dict['fifa_index'])):
        info_dict["rating"].append(df[(df['long_name'] == info_dict['long_name'][i])]['overall'][info_dict['fifa_index'][i]])

    info_dict['short_name'] = [short_name for short_name in classification['Player']]

    for i in range(len(info_dict['fifa_index'])):
        info_dict['points'].append(classification[(classification['Player'] == info_dict['short_name'][i])]['Points'][i])

    merged_info = pd.DataFrame(info_dict)
    merged_info = merged_info.sort_values(by=['fifa_index'])
    merged_info = merged_info.reset_index(drop = True)
    df = df.join(merged_info['points'])

    return df

In [39]:
# We run a test of the function to see if the points column has been appended. 
# Also, we will print the classification points below to verify the order!
test = points_column_append(fifa_ratings_2017, top_10_2017, classification_2017)
test

,player_id,fifa_version,short_name,long_name,player_positions,overall,value_eur,wage_eur,age,height_cm,...,dribbling,defending,physic,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,points
0,20801,17,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,LW,94,87000000.0,575000.0,31,185,...,91.0,33.0,80.0,7,11,15,14,11,NaN,946
1,158023,17,L. Messi,Lionel Andrés Messi Cuccittini,RW,93,89000000.0,575000.0,29,170,...,96.0,26.0,61.0,6,11,15,14,8,NaN,670
2,190871,17,Neymar,Neymar da Silva Santos Júnior,LW,92,109000000.0,450000.0,24,174,...,95.0,30.0,56.0,9,9,15,15,11,NaN,361
3,188545,17,R. Lewandowski,Robert Lewandowski,ST,90,71000000.0,250000.0,27,185,...,85.0,38.0,82.0,15,6,12,8,10,NaN,45
4,155862,17,Sergio Ramos,Sergio Ramos García,CB,89,43500000.0,300000.0,30,183,...,71.0,87.0,83.0,11,8,9,7,11,NaN,71
5,177003,17,L. Modrić,Luka Modrić,CM,89,54000000.0,325000.0,30,174,...,89.0,72.0,66.0,13,9,7,14,9,NaN,84
6,1179,17,G. Buffon,Gianluigi Buffon,GK,88,9000000.0,120000.0,38,192,...,NaN,NaN,NaN,87,88,68,90,84,46.0,221
7,202126,17,H. Kane,Harry Kane,ST,84,41000000.0,140000.0,22,188,...,78.0,42.0,81.0,8,10,11,14,11,NaN,36
8,215914,17,N. Kanté,N'Golo Kanté,CM,81,21000000.0,130000.0,25,169,...,77.0,81.0,83.0,15,12,10,7,10,NaN,47
9,231747,17,K. Mbappe Lottin,Kylian Mbappé Lottin,LW,71,3100000.0,20000.0,17,178,...,74.0,39.0,62.0,13,5,7,11,6,NaN,48


In [40]:
fifa_ratings_2017 = points_column_append(fifa_ratings_2017, top_10_2017, classification_2017)
fifa_ratings_2017

,player_id,fifa_version,short_name,long_name,player_positions,overall,value_eur,wage_eur,age,height_cm,...,dribbling,defending,physic,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,points
0,20801,17,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,LW,94,87000000.0,575000.0,31,185,...,91.0,33.0,80.0,7,11,15,14,11,NaN,946
1,158023,17,L. Messi,Lionel Andrés Messi Cuccittini,RW,93,89000000.0,575000.0,29,170,...,96.0,26.0,61.0,6,11,15,14,8,NaN,670
2,190871,17,Neymar,Neymar da Silva Santos Júnior,LW,92,109000000.0,450000.0,24,174,...,95.0,30.0,56.0,9,9,15,15,11,NaN,361
3,188545,17,R. Lewandowski,Robert Lewandowski,ST,90,71000000.0,250000.0,27,185,...,85.0,38.0,82.0,15,6,12,8,10,NaN,45
4,155862,17,Sergio Ramos,Sergio Ramos García,CB,89,43500000.0,300000.0,30,183,...,71.0,87.0,83.0,11,8,9,7,11,NaN,71
5,177003,17,L. Modrić,Luka Modrić,CM,89,54000000.0,325000.0,30,174,...,89.0,72.0,66.0,13,9,7,14,9,NaN,84
6,1179,17,G. Buffon,Gianluigi Buffon,GK,88,9000000.0,120000.0,38,192,...,NaN,NaN,NaN,87,88,68,90,84,46.0,221
7,202126,17,H. Kane,Harry Kane,ST,84,41000000.0,140000.0,22,188,...,78.0,42.0,81.0,8,10,11,14,11,NaN,36
8,215914,17,N. Kanté,N'Golo Kanté,CM,81,21000000.0,130000.0,25,169,...,77.0,81.0,83.0,15,12,10,7,10,NaN,47
9,231747,17,K. Mbappe Lottin,Kylian Mbappé Lottin,LW,71,3100000.0,20000.0,17,178,...,74.0,39.0,62.0,13,5,7,11,6,NaN,48


In [41]:
# Returns True! We can proudly announce that the dataframe has been successfully cleaned 🎉
print(list(test["points"])) == print(list(classification_2017["Points"]))

[946, 670, 361, 45, 71, 84, 221, 36, 47, 48]
[946, 670, 361, 221, 84, 71, 48, 47, 45, 36]


True

In [42]:
# Last, we append points colum to every FIFA ratings dataset.
fifa_ratings_2018 = points_column_append(fifa_ratings_2018, top_10_2018, classification_2018)
fifa_ratings_2019 = points_column_append(fifa_ratings_2019, top_10_2019, classification_2019)
fifa_ratings_2021 = points_column_append(fifa_ratings_2021, top_10_2021, classification_2021)
fifa_ratings_2022 = points_column_append(fifa_ratings_2022, top_10_2022, classification_2022)

We make the last two arrangements before exporting datasets:
- Define de general position of the player: Attacker, Midfielder, Defender or Goalkeeper;
- Separate players from goalkeepers, since the metrics of interest are different.

In [43]:
# Function that appends a new column with a generic player position:
def general_position (df):
    """
    Takes the dataframe as argument and appends a new column with a generic position according
    to 'players_positions'.
    """
    # We define an empty list and three generic lists including all positions in the datasets.
    general_position = []
    attacker = ['LW', 'RW', 'ST', 'CF']
    midfielder = ['CM', 'CAM', 'CDM']
    defender = ['CB']
    goalkeeper = ['GK']

    # We iterate through every 'player_position' and append the generic one to the list.
    for position in df['player_positions']:
        if position in attacker:
            general_position.append('ATT')
        elif position in midfielder:
            general_position.append('MID')
        elif position in defender:
            general_position.append('DEF')
        elif position in goalkeeper:
            general_position.append('GK')

    # We append the new column to the dataframe.
    df.insert(5, 'general_position', general_position)
    return df

In [44]:
fifa_ratings_2017 = general_position(fifa_ratings_2017)
fifa_ratings_2018 = general_position(fifa_ratings_2018)
fifa_ratings_2019 = general_position(fifa_ratings_2019)
fifa_ratings_2021 = general_position(fifa_ratings_2021)
fifa_ratings_2022 = general_position(fifa_ratings_2022)

In [45]:
# Now we filter by GK's:
def goalkeeper_df (df):
    """
    Function that filters by GK condition
    Accepts the dataframe as argument and 
    Returns the new dataframe of GK's while removes the GK from the original df.
    """
    fifa_gk = df[(df['player_positions'] == "GK")]
    fifa_gk.reset_index(drop = True, inplace = True)
    return fifa_gk

In [46]:
fifa_gk17 = goalkeeper_df(fifa_ratings_2017)
fifa_gk18 = goalkeeper_df(fifa_ratings_2018)
fifa_gk19 = goalkeeper_df(fifa_ratings_2019)
fifa_gk21 = goalkeeper_df(fifa_ratings_2021)
fifa_gk22 = goalkeeper_df(fifa_ratings_2022)

Since there are a few goalkeepers, we concatenate all of GK's dataframes:

In [47]:
fifa_gk = pd.concat([fifa_gk17, fifa_gk18, fifa_gk19, fifa_gk21, fifa_gk22], ignore_index=True, sort=False)
fifa_gk

,player_id,fifa_version,short_name,long_name,player_positions,general_position,overall,value_eur,wage_eur,age,...,dribbling,defending,physic,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,points
0,1179,17,G. Buffon,Gianluigi Buffon,GK,GK,88,9000000.0,120000.0,38,...,NaN,NaN,NaN,87,88,68,90,84,46.0,221
1,212831,19,Alisson,Alisson Ramsés Becker,GK,GK,85,36500000.0,110000.0,25,...,NaN,NaN,NaN,82,81,85,83,88,49.0,67
2,230621,21,G. Donnarumma,Gianluigi Donnarumma,GK,GK,85,93500000.0,34000.0,21,...,NaN,NaN,NaN,89,81,76,83,89,50.0,36
3,192119,22,T. Courtois,Thibaut Nicolas Marc Courtois,GK,GK,89,85500000.0,250000.0,29,...,NaN,NaN,NaN,84,89,74,86,88,46.0,82


In [48]:
# After this step, we remove GK's from the FIFA ratings dataframe, where we will only keep field players.
def GK_remover_df (fifa_ratings_df):
    """
    Function that removes instances of GoalKeepers from the FIFA ratings dataframes, since the original 
    funct was unable to perform this utility apart from filtering the origingal df to create another one just for GK'set
    Accepts a FIFA ratings df
    Returns it without GK instances
    """
    # Define removing condition to apply it to the df
    GK_condition = fifa_ratings_df[(fifa_ratings_df['player_positions'] == "GK")].index
    fifa_ratings_df.drop(GK_condition, inplace = True)
    return fifa_ratings_df

In [49]:
fifa17 = GK_remover_df(fifa_ratings_2017)
fifa18 = GK_remover_df(fifa_ratings_2018)
fifa19 = GK_remover_df(fifa_ratings_2019)
fifa21 = GK_remover_df(fifa_ratings_2021)
fifa22 = GK_remover_df(fifa_ratings_2022)

We export the 6 dataframes and close this part of the project.


In [50]:
fifa17.to_csv("../csv_files/fifa17.csv") 
fifa18.to_csv("../csv_files/fifa18.csv")
fifa18.to_csv("../csv_files/fifa19.csv")
fifa21.to_csv("../csv_files/fifa21.csv")
fifa22.to_csv("../csv_files/fifa22.csv")
fifa_gk.to_csv("../csv_files/fifagk.csv")

In [58]:
fifa_20 = year_filter (ratings, 20)
fifa_20 = GK_remover_df(fifa_20)
fifa_20 = fifa_20.head(30)

#fifa_20.describe()

/var/folders/lr/stnkft6j5ps6pysrw7v0cgp80000gn/T/ipykernel_4799/998472450.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fifa_ratings_df.drop(GK_condition, inplace = True)


In [ ]:
fifa_20.to_csv("../csv_files/fifagk.csv")

In [59]:
fifa_20

,player_id,fifa_version,short_name,long_name,player_positions,overall,value_eur,wage_eur,age,height_cm,...,passing,dribbling,defending,physic,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
0,158023,20,L. Messi,Lionel Andrés Messi Cuccittini,"RW, CF, ST",94,95500000.0,560000.0,32,170,...,92.0,96.0,39.0,66.0,6,11,15,14,8,NaN
1,20801,20,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",93,58500000.0,410000.0,34,187,...,82.0,89.0,35.0,78.0,7,11,15,14,11,NaN
2,190871,20,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",92,105500000.0,290000.0,27,175,...,87.0,95.0,32.0,58.0,9,9,15,15,11,NaN
3,183277,20,E. Hazard,Eden Michael Hazard,"LW, CF",91,90000000.0,470000.0,28,175,...,86.0,94.0,35.0,66.0,11,12,6,8,8,NaN
4,192985,20,K. De Bruyne,Kevin De Bruyne,"CAM, CM",91,90000000.0,370000.0,28,181,...,92.0,86.0,61.0,78.0,15,13,5,10,13,NaN
6,177003,20,L. Modrić,Luka Modrić,CM,90,45000000.0,340000.0,33,172,...,89.0,89.0,72.0,66.0,13,9,7,14,9,NaN
8,203376,20,V. van Dijk,Virgil van Dijk,CB,90,78000000.0,200000.0,27,193,...,70.0,71.0,90.0,86.0,13,10,13,11,11,NaN
9,209331,20,M. Salah,Mohamed Salah Ghaly,"RW, ST",90,80500000.0,240000.0,27,175,...,81.0,89.0,45.0,74.0,14,14,9,11,14,NaN
10,138956,20,G. Chiellini,Giorgio Chiellini,CB,89,24500000.0,210000.0,34,187,...,58.0,60.0,90.0,82.0,3,3,2,4,3,NaN
11,153079,20,S. Agüero,Sergio Leonel Agüero del Castillo,ST,89,60000000.0,300000.0,31,173,...,77.0,88.0,33.0,74.0,13,15,6,11,14,NaN
